In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from adjustText import adjust_text

from PIL import Image
from pdb import set_trace as st 
import random
import json
import os 
from tqdm import tqdm
import string
from src.Aux import * 
from src.Triangles import Triangle
from src.Illustrate import * 
from src.text_in_bbox import *

In [2]:
emnist_loc = "/home/yfrid/Desktop/stem-whiteboard/dataset/mnist/EMNIST/raw/"
emnist_images = read_idx_ubyte(emnist_loc+"emnist-byclass-train-images-idx3-ubyte")
emnist_labels = read_idx_ubyte(emnist_loc+"emnist-byclass-train-labels-idx1-ubyte")
emnist_chars = string.digits+string.ascii_uppercase+string.ascii_lowercase
EMNIST = EMNIST_Handler(emnist_images,emnist_labels,emnist_chars)

Reading image data: 697932 images of 28x28 pixels.
Reading label data: 697932 labels.


/home/yfrid/Desktop/GitProjects/whiteboard-dataset/src/l2q_inference.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load(weights_path,

In [3]:
lower_letters = string.ascii_lowercase
#greek_letters = ['α', 'β', 'γ', 'δ', 'θ']

def plot_geometry_improved(description):
    fig, ax = plt.subplots(figsize=(6,6),dpi = 64)
    plt.subplots_adjust(left=0, right=1, bottom=0, top=1)
    labels = []
    handwritten =bool(random.getrandbits(1))
    # Set up the plot with some padding
    all_x = [v["x"] for v in description["vertices"]]
    all_y = [v["y"] for v in description["vertices"]]
    x_range = max(all_x) - min(all_x)
    y_range = max(all_y) - min(all_y)
    padding = max(x_range, y_range) * 0.1
    
    ax.set_xlim(min(all_x) - padding, max(all_x) + padding)
    ax.set_ylim(min(all_y) - padding, max(all_y) + padding)
####################################################### SEGMENTS
    description["index_lookup"] = {d["mark"]: i for i, d in enumerate(description["vertices"])} ## add index to each vertice for easier search
    description,segment_labels = plot_segments(description,ax,EMNIST,handwritten)
################################################### SPECIALS
    description = plot_specials(description,ax,EMNIST,handwritten)

################################################### ANGLES
    description, angle_labels = plot_angles(description,ax,EMNIST,handwritten)


#################################################### Text

     
    angles = [a["mark"] for a in description["angles"]]
    segments = [s["mark"] for s in description["segments"]]
    angle_questions = []
    segment_questions = []
    for question in description["questions"]:
        angle = False
        for a in angles:
            if a in question:
                angle_questions.append(a)
                angle = True
        if angle == True:
            continue
        for s in segments:
            if s in question:
                segment_questions.append(s)




    
    x_com,y_com = 0,0
    for v in description["vertices"][:3]: #### find COM of triangle using first three vertices    
        x_com+=v["x"]/3
        y_com+=v["y"]/3
    
    for segment in description["segments"]:
        if segment["known"] == True or segment["mark"] in segment_questions:
            inds = [description["index_lookup"][mark] for mark in segment["mark"]]
            A, B = [np.array([description["vertices"][ind]["x"],description["vertices"][ind]["y"]],dtype = np.float64) for ind in inds]
            AB = B - A 
            center = A + (0.5+0.1*np.random.rand())*AB
            AB_perp = np.array([AB[1],-AB[0]])
            outer_vec = center - np.array([x_com,y_com]) ### a vector pointing out of the triangle
            if np.dot(AB_perp,outer_vec) <0: ### if they are pointing opposite direction, reverse perp
                AB_perp *= -1
            AB_perp *= 0.1*np.linalg.norm(AB)/np.linalg.norm(AB_perp)
            AB_perp_size = np.linalg.norm(AB_perp)
            if handwritten:
                if segment["mark"] in segment_questions:
                    mat = EMNIST.token_mat("?")
                else:      
                    mat = EMNIST.token_mat(str(segment["length"]))
                x_extra = mat.shape[1]/mat.shape[0]- 1
                text_center = center + AB_perp
                xs = text_center[0]+0.5*np.array([-(1+x_extra),1+x_extra])*AB_perp_size
                ys = text_center[1]+np.array([-0.5,0.5])*AB_perp_size
                while max(xs) > center[0] and min(xs) < center[0] and max(ys) > center[1] and min(ys) < center[1]:
                    xs[0]+= 0.1*AB_perp_size*AB_perp[0]
                    xs[1]+= 0.1*AB_perp_size*AB_perp[0]
                    ys[0]+= 0.1*AB_perp_size*AB_perp[1]
                    ys[1]+= 0.1*AB_perp_size*AB_perp[1]
                
                lims = list(ax.get_xlim())
                if xs[1] > lims[1]:
                    lims[1] = 2*xs[1]-lims[1]
                if xs[0] < lims[0]:
                    lims[0] = lims[0] - abs(lims[0]-xs[0])
                ax.set_xlim(lims)
                lims = list(ax.get_ylim())
                if ys[1] > lims[1]:
                    lims[1] = 2*ys[1]-lims[1]
                if ys[0] < lims[0]:
                    lims[0] = lims[0] - abs(lims[0]-ys[0])
                ax.set_ylim(lims)
                plt.imshow(mat,cmap='gray',extent=[min(xs),max(xs),min(ys),max(ys)],
                        vmin=0,
                        vmax=1,
                        alpha=1 # Transparency (0=invisible, 1=opaque)
                    )                
            else:
                if segment["mark"] in segment_questions:
                    plt.text((center+AB_perp)[0],(center+AB_perp)[1],"?")
                else:
                    plt.text((center+AB_perp)[0],(center+AB_perp)[1],segment["length"])


    for angle in description["angles"]:
        if angle["known"] == True or angle["mark"] in angle_questions:
            inds = [description["index_lookup"][mark] for mark in angle["mark"]]
            A,B,C = [np.array([description["vertices"][ind]["x"],description["vertices"][ind]["y"]],dtype = np.float64) for ind in inds]
            BA = A-B 
            BC = C-B
            BA_size,BC_size = np.linalg.norm(BA),np.linalg.norm(BC)
            radius = 0.2*min(BA_size,BC_size)
            bisector = (BA/BA_size+BC/BC_size)
            bisector *= radius/np.linalg.norm(bisector)
            points = [B,B+BA*radius/BA_size,B+bisector,B+BC*radius/BC_size] 
            center = np.zeros(2)
            for point in points:
                center+= point/4
            if handwritten:
                if angle["mark"] in angle_questions:
                    mat = EMNIST.token_mat("?")
                else:
                    mat = EMNIST.token_mat(str(angle["value"]))
                R = mat.shape[0]/float(mat.shape[1])
                corners = largest_axis_aligned_rect_in_kite(points, R, tol=1e-9)["corners"]                
#                vectors = np.array(points)-center
#                radiuses = np.linalg.norm(vectors,axis = 1)
#                radius_vector = vectors[np.argmin(radiuses)]
                plt.imshow(mat,extent = [corners[0][0],corners[2][0],corners[0,1],corners[2][1]],cmap = 'gray',vmin = 0, vmax = 1, alpha = 1)
            else:
                width = max([i[0] for i in points]) - min([i[0] for i in points])
                if angle["mark"] in angle_questions:
                    plt.text(center[0]-width/3,center[1],'?',ha = 'left')
                else:
                    plt.text(center[0]-width/3,center[1],angle["value"],ha = 'left')

    #################################################### VERTICES    
    labels = plot_vertices(description,ax,EMNIST,handwritten)
    
    for v in description["vertices"]:
        v["x"] = np.around((v["x"]-min(all_x))/(max(all_x)-min(all_x)),2)
        v["y"] = np.around((v["y"]-min(all_y))/(max(all_y)-min(all_y)),2)
        

##############################################################    
    # Combine all labels for adjustment
    all_labels = labels + segment_labels + angle_labels
    
    # Get all non-text artists (points, lines) to avoid
    avoid_objects = [artist for artist in ax.get_children() 
                    if not isinstance(artist, plt.Text)]
    
    # Fine-tune the label positioning
    adjust_text(all_labels, ax=ax,
                add_objects=avoid_objects,
                expand_points=(1.5, 1.5),    # Distance from points
                expand_text=(1.2, 1.2),     # Distance between labels
                expand_objects=(1.2, 1.2),  # Distance from lines/objects
                arrowprops=dict(arrowstyle='-', color='gray', alpha=0.5, lw=0.5),
                force_points=(0.5, 0.5),    # Force to avoid points
                force_text=(0.5, 0.5),      # Force to avoid text overlap
                force_objects=(0.3, 0.3),   # Force to avoid other objects
                lim=1000)                   # Maximum iterations
    
    ax.set_aspect('equal')
    ax.axis('off')

    return description


def convert_numpy_types(obj):
    if isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, (np.int_, np.intc, np.intp, np.int8, np.int16, np.int32, np.int64)):
        return int(obj)
    elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64)):
        return float(obj)
    elif isinstance(obj, dict):
        return {k: convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [convert_numpy_types(v) for v in obj]
    else:
        return obj



In [4]:
def save_json(description, output_dir, idx):
    os.makedirs(f"{output_dir}/labels", exist_ok=True)
    with open(f"{output_dir}/labels/{idx:06d}.json", 'w') as f:
        json.dump(description, f, indent=2)  # indent=2 for human readability
        
np.random.seed(10)#np.random.randint(100))  # For reproducible results
output_dir = "/home/yfrid/Desktop/stem-whiteboard/dataset/4V"


for idx in tqdm(range(1500,1800)):
    tri = Triangle(np.random.choice(range(5,15)), #### Random limb A 
                   np.random.choice(range(5,15)), #### Random limb B
                   rotation=np.random.randint(360),#### Random rotation
                   mirror=np.random.choice([True,False])) #### random mirroring 
    third_vertice = np.random.choice(["scalene","right"])
    tri.third_vertice(third_vertice)    
#    tri.set_4V_bisector_question(num_questions = np.random.choice(range(1,3)))
#    tri.set_4V_median_question(num_questions = np.random.choice(range(1,3)))
    tri.set_4V_altitude_question(num_questions = np.random.choice(range(1,3)))    tri.rotate()
    new = plot_geometry_improved(tri.description)
    plt.savefig(f"{output_dir}/images/{idx:06d}.png", dpi=64,facecolor='white')
    save_json(convert_numpy_types(new), output_dir, idx)
    
    plt.close()


100%|█████████████████████████████████████████| 300/300 [00:35<00:00,  8.43it/s]


In [5]:
def save_json(description, output_dir, idx):
    os.makedirs(f"{output_dir}/labels", exist_ok=True)
    with open(f"{output_dir}/labels/{idx:06d}.json", 'w') as f:
        json.dump(description, f, indent=2)  # indent=2 for human readability
        
np.random.seed(15)#np.random.randint(100))  # For reproducible results
output_dir = "/home/yfrid/Desktop/stem-whiteboard/dataset/4V"


for idx in tqdm(range(1500,1800)):
    tri = Triangle(np.random.choice(range(5,15)), #### Random limb A 
                   np.random.choice(range(5,15)), #### Random limb B
                   rotation=np.random.randint(360),#### Random rotation
                   mirror=np.random.choice([True,False])) #### random mirroring 
    third_vertice = np.random.choice(["scalene","right"])
    tri.third_vertice(third_vertice)    
#    tri.set_4V_bisector_question(num_questions = np.random.choice(range(1,3)))
#    tri.set_4V_median_question(num_questions = np.random.choice(range(1,3)))
    tri.set_4V_altitude_question(num_questions = np.random.choice(range(1,3)))
    tri.rotate()
    new = plot_geometry_improved(tri.description)
    plt.savefig(f"{output_dir}/images/{idx:06d}.png", dpi=64,facecolor='white')
    save_json(convert_numpy_types(new), output_dir, idx)
    
    plt.close()


100%|█████████████████████████████████████████| 300/300 [00:34<00:00,  8.61it/s]
